# Search using query rules

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/05-query-rules.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to how use query rules, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
You'll store query rules in Elasticsearch using the [query rules API](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-rules-apis.html) and query them using [rule_query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-rule-query.html).

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

Once logged in to your Elastic Cloud account, go to the [Create deployment](https://cloud.elastic.co/deployments/create) page and select **Create deployment**. Make sure the Elasticsearch version is **8.10.0** or newer. Leave all other settings with their default values.

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [1]:
!pip install -qU elasticsearch

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [2]:
from elasticsearch import Elasticsearch
from getpass import getpass

CLOUD_ID = getpass("Elastic Cloud ID")
CLOUD_PASSWORD = getpass("Elastic Password")

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", CLOUD_PASSWORD)
)

Elastic Cloud ID ········
Elastic Password ········


If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally.

Confirm that the client has connected with this test.

In [3]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': 'ad402eb9a59041458b8edfc021e91caf', 'cluster_uuid': 'ks_HfcCdSf2qrcKZQsk9Lg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of products with the following fields:

- `name`
- `description`
- `price`
- `currency`
- `plug_type`
- `voltage`

### Index test data

Run the following command to upload some sample data.

In [4]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/query-rules-data.json"
response = urlopen(url)
docs = json.loads(response.read())

operations = []
for doc in docs:
    operations.append({"index": {"_index": "products_index", "_id": doc["id"]}})
    operations.append(doc["content"])
client.bulk(index="products_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 5, 'items': [{'index': {'_index': 'products_index', '_id': 'us1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 4, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'products_index', '_id': 'uk1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 5, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'products_index', '_id': 'eu1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 6, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'products_index', '_id': 'preview1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 7, '_primary_term': 1, 'status': 200}}]})

## Search the test data

First, let's search our data for a reliable wireless charger.

Before we search our data, we'll define some convenience functions to output the raw JSON responses from Elasticsearch into a more human-readable format.

In [5]:
def pretty_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response['hits']['hits']:
            id = hit['_id']
            score = hit['_score']
            name = hit['_source']['name']
            description = hit['_source']['description']
            price = hit["_source"]["price"]
            currency = hit["_source"]["currency"]
            plug_type = hit["_source"]["plug_type"]
            voltage = hit["_source"]["voltage"]
            pretty_output = (f"\nID: {id}\nName: {name}\nDescription: {description}\nPrice: {price}\nCurrency: {currency}\nPlug type: {plug_type}\nVoltage: {voltage}\nScore: {score}")
            print(pretty_output)

def pretty_ruleset(response):
    print("Ruleset ID: " + response['ruleset_id'])
    for rule in response['rules']:
        rule_id = rule['rule_id']
        type = rule['type']
        print(f"\nRule ID: {rule_id}\n\tType: {type}\n\tCriteria:")
        criteria = rule['criteria']
        for rule_criteria in criteria:
            criteria_type = rule_criteria['type']
            metadata = rule_criteria['metadata']
            values = rule_criteria['values']
            print(f"\t\t{metadata} {criteria_type} {values}")
        ids = rule['actions']['ids']
        print(f"\tPinned ids: {ids}")

Next, do the search:

In [6]:
response = client.search(index="products_index", query={
    "multi_match": {
        "query": "reliable wireless charger for iPhone",
        "fields": [ "name^5", "description" ]
    }
})

pretty_response(response)


ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.0668641

ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it e

As we can see from the response, the European result is ranked first. This might not be desirable if, for example, I know that my searcher is coming from the US or the UK which have different plugs and specifications. 

Query rules can help here!

## Creating rules

Let's assume that separately, we know what country our users are coming from (perhaps geolocation from IP addresses or logged in user account information). Now, we want to create query rules to boost wireless chargers based on that information when people search for anything containing the phrase `wireless charger`.

In [7]:
client.query_ruleset.put(ruleset_id="promotion-rules", rules=[
    {
      "rule_id": "us-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["us"]
        }
      ],
      "actions": {
        "ids": [
          "us1"
        ]
      }
    },
    {
      "rule_id": "uk-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["uk"]
        }
      ],
      "actions": {
        "ids": [
          "uk1"
        ]
      }
    }
  ])

ObjectApiResponse({'result': 'updated'})

In order for these rules to match, one of the following must be true:

- `my_query` contains the string "wireless charger" *AND* `country` is "us"
- `my_query` contains the string "wireless charger" *AND* `country` is "uk"

We can view our ruleset using the API as well (with another `pretty_ruleset` function for readability):

In [8]:
response = client.query_ruleset.get(ruleset_id="promotion-rules")
pretty_ruleset(response)

Ruleset ID: promotion-rules

Rule ID: us-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['us']
	Pinned ids: ['us1']

Rule ID: uk-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['uk']
	Pinned ids: ['uk1']


Next, we use the rule_query to perform a search using the same organic query as above, but with the addition of query rules:

In [9]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "us"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it ensures your devices are powered swiftly and safely. The future of charging is here.
Price: 15.0
Currency: USD
Plug type: B
Voltage: 120v
Score: 1.7014122e+38

ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to cha

The rule query boosts the documents that we want to be displayed first.

Note that all criteria in a rule must match in order for a rule to be applied. If we update the `country` to be "ca" for example, neither query rule will be applied and we will return the organic, unmodified result set.

In [10]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "ca"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.0668641

ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it e

It's also possible for multiple rules to apply to a single rule query. Let's update our ruleset, to always pin a promotional result for a pre-order page for our newest model. 

Because rules are applied in order, we'll put the pre-order document at the beginning of the ruleset.

In [11]:
client.query_ruleset.put(ruleset_id="promotion-rules", rules=[
    {
      "rule_id": "preorder",
      "type": "pinned",
      "criteria": [
        {
          "type": "always"
        }
      ],
      "actions": {
        "ids": [
          "preview1"
        ]
      }
    },
    {
      "rule_id": "us-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["us"]
        }
      ],
      "actions": {
        "ids": [
          "us1"
        ]
      }
    },
    {
      "rule_id": "uk-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["uk"]
        }
      ],
      "actions": {
        "ids": [
          "uk1"
        ]
      }
    }
  ])

ObjectApiResponse({'result': 'updated'})

Now, doing a search from the UK will pin the pre-order page first, then pin the UK result second, before returning the organic search results.

In [12]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "uk"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.7014124e+38

ID: uk1
Name: PureJuice Pro - UK Compatible
Description: PureJuice Pro: Redefining wireless charging. Seamlessly merging swift charging capabilities with a refined aesthetic, it guarantees your devices receive rapid and secure power. Welcome to the next generation of charging.
Price: 20.0
Currency: GBP
Plug type: G
Voltage: 230V
Score: 1.7014122e+38

ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled chargin

These examples start to show the power of promoting documents based on contextual query metadata. For more information on how to get started using query rules, check out our [blog post](https://www.elastic.co/blog/introducing-query-rules-elasticsearch-8-10) and [search guide](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-using-query-rules.html).